In [1]:
import dart_fss as dart
import pandas as pd

In [2]:
df_list = pd.read_csv('data/c_list.csv')

In [3]:
df_list = df_list['name'].tolist()
print(df_list)

['데코앤이', '신텍', '금오하이텍', '에이치앤아이', '티에스엠텍', '우양에이치씨', '배명금속', '금강제강', '풍림산업', '미주제강', '제이콤', '인네트', '신세계톰보이', '에스피코프', '쌈지', '카라반케이디이', '유리이에스', '아이비진', '우영', '알루코', '케이티씨텔레콤', '아이텍스필', '현주컴퓨터', '넥스텔', '지에스엔텍', '현대멀티캡', '중앙제지', '테크메이트', '텔슨전자', '금강화섬', '호성', '대호', '모닷텔', '화림모드', '서통', '계몽사', '코리아링크', '시스컴', '정산애강', '소프트윈', '심스밸리', '코닉스', '유니씨앤티', '아이씨켐', '디지텔', '삼한콘트롤스', '피어리스', '정일이엔씨', '삼산', '교하산업', '아진산업', '주화산업', '라인건설', '대륭산업', '한국종합철관']


In [ ]:
api_key='f60bb69488d19533e2bdb54a05343f20979e4307'
dart.set_api_key(api_key)
crp_list = dart.get_corp_list()

In [ ]:
class search_company():

    def search_table(cmp_list):
        basic_info = crp_list.find_by_corp_name(cmp_list, exactly=True)[0]
        fs = basic_info.extract_fs(bgn_de='20190101')

        # 'bs' 재무상태표, 'is' 손익계산서, 'cis' 포괄손익계산서, 'cf' 현금흐름표
        df_cis = fs['cis']
        df_bs = fs['bs']
        df_cf = fs['cf']

        df_cis.to_csv('data/포괄손익계산서/' + cmp_list + ' 포괄손익계산서.csv', index=False)
        df_bs.to_csv('data/재무상태표/' + cmp_list + ' 재무상태표.csv', index=False)
        df_cf.to_csv('data/현금흐름도/' + cmp_list + ' 현금흐름표.csv', index=False)

        return df_cis, df_bs, df_cf

    def cis_preprocessing(df_cis):
        df_cis_1 = df_cis.iloc[:, [1, 7, 8, 9]]
        df_cis_1.columns = ['', df_cis_1.columns[1][0], df_cis_1.columns[2][0], df_cis_1.columns[3][0]]
        df_cis_1 = df_cis_1.T
        df_cis_1 = df_cis_1.rename(columns=df_cis_1.iloc[0])
        df_cis_1 = df_cis_1.drop(df_cis_1.index[0])
        df_cis_1 = df_cis_1.reset_index().rename(columns={'index': 'date'})
        df_cis_1['date'] = df_cis_1['date'].str[9:]
        print('cis_proprocessing complete')

        return df_cis_1

    def bs_preprocessing(df_bs):
        df_bs_1 = df_bs.iloc[:, [1, 7, 8, 9]]
        df_bs_1.columns = ['', df_bs_1.columns[1][0], df_bs_1.columns[2][0], df_bs_1.columns[3][0]]
        df_bs_1 = df_bs_1.T
        df_bs_1 = df_bs_1.rename(columns=df_bs_1.iloc[0])
        df_bs_1 = df_bs_1.drop(df_bs_1.index[0])
        df_bs_1 = df_bs_1.reset_index().rename(columns={'index': 'date'})
        print('bs_preprocessing complete')

        return df_bs_1

    def cf_preprocessing(df_cf):
        df_cf_1 = df_cf.iloc[:, [1, 6, 7, 8]]
        df_cf_1.columns = ['', df_cf_1.columns[1][0], df_cf_1.columns[2][0], df_cf_1.columns[3][0]]
        df_cf_1 = df_cf_1.T
        df_cf_1 = df_cf_1.rename(columns=df_cf_1.iloc[0])
        df_cf_1 = df_cf_1.drop(df_cf_1.index[0])
        df_cf_1 = df_cf_1.reset_index().rename(columns={'index': 'date'})
        df_cf_1['date'] = df_cf_1['date'].str[9:]
        print('cf_preprocessing complete')

        return df_cf_1

    def total_dataframe(self, df_cis_1, df_bs_1, df_cf_1, cmp_list):
        data_total = pd.merge(df_cis_1, df_bs_1, on='date')
        data_total = pd.merge(data_total, df_cf_1, on='date')
        data_total.to_csv('data/total/' + cmp_list + ' total.csv', index=False)

        return data_total

In [15]:
if '__main__' == __name__:
    ok_list = []
    no_list = []
#     result = pd.DataFrame(index=range(0,0), columns=['date', 'company', '당기순이익(손실)', '기타포괄손익', '후속적으로 당기손익으로 재분류될 수 있는 항목', '해외사업장환산외환차이', '지분법자본변동', '매도가능금융자산의 평가손익', '후속적으로 당기손익으로 재분류되지 않는 항목', '순확정급여부채의 재측정요소', '기타포괄손익-공정가치 측정 금융자산평가이익', '당기총포괄손익', '"총 포괄손익', ' 지배기업의 소유주에게 귀속되는 지분"', '"총 포괄손익', ' 비지배지분"', '유동자산', '현금및현금성자산', '단기금융상품', '매출채권', '기타채권', '재고자산', '유동계약자산', '당기손익-공정가치 측정 금융자산', '기타유동자산', '매각예정비유동자산', '비유동자산', '장기금융상품', '장기기타채권', '기타포괄손익-공정가치 측정 금융자산', '유형자산', '무형자산', '투자부동산', '사용권자산', '공동기업투자', '기타비유동자산', '자산총계', '유동부채', '단기차입금', '매입채무', '기타채무', '유동성리스부채', '계약부채', '전환사채', '파생상품부채', '기타유동부채', '기타충당부채', '비유동부채', '순확정급여부채', '비유동리스부채', '기타비유동부채', '부채총계', '지배기업의 소유주에게 귀속되는 자본', '자본금', '주식발행초과금', '기타포괄손익누계액', '기타자본항목', '이익잉여금(결손금)', '비지배지분', '자본총계', '자본과부채총계', '영업활동현금흐름', '영업에서 창출된 현금흐름', '이자수취', '이자지급', '배당금수취(영업)', '법인세의 환급(납부)', '투자활동현금흐름', '장기금융상품의 감소', '단기금융상품의 감소', '기타채권의 대여', '기타채권의 회수', '당기손익-공정가치 측정 금융자산의 처분', '당기손익-공정가치 측정 금융자산의 취득', '기타포괄손익-공정가치 측정 금융자산 처분', '투자부동산의 취득', '유형자산의 처분', '유형자산의 취득', '무형자산의 취득', '무형자산의 처분', '임차보증금의 증가', '임차보증금의 감소', '공동기업의 투자', '매각예정비유동자산처분', '연결범위변동으로 인한 현금의 증감', '재무활동현금흐름', '전환사채의 발행', '전환사채의 상환', '단기차입금의 차입', '단기차입금의 상환', '리스부채의 감소', '유상증자', '주식발행비', '비지배분의 순증감', '현금및현금성자산의 순증감', '기초현금및현금성자산', '현금및현금성자산에 대한 환율변동효과', '기말현금및현금성자산'])
#     cm = company_module.search_company()
    for cmp_list in df_list:
        try:
            df_cis, df_bs, df_cf = search_company.search_table(cmp_list)
            df_cis_1 = search_company.cis_preprocessing(df_cis)
            df_bs_1 = search_company.bs_preprocessing(df_bs)
            df_cf_1 = search_company.cf_preprocessing(df_cf)
            
            total = search_company.total_dataframe(df_cis_1, df_bs_1, df_cf_1, cmp_list)
            
            total['company'] = cmp_list
#             result = pd.concatcat([result, total])
            ok_list.append(cmp_list)
        except:
            no_list.append(cmp_list)

    df1 = pd.DataFrame(ok_list, columns=['company'])
    df2 = pd.DataFrame(no_list, columns=['company'])
    df1.to_csv('data/검색가능.csv')
    df2.to_csv('data/검색불가.csv')
#     result.to_csv('data/result_total.csv', index=False)



Output()

Output()


cis_proprocessing complete
bs_preprocessing complete
cf_preprocessing complete


Output()

Output()

...

Output()

Output()

Output()

Output()

Output()

In [179]:
# api_key='f60bb69488d19533e2bdb54a05343f20979e4307'
# dart.set_api_key(api_key)
# crp_list = dart.get_corp_list()

# class search_company():
#     def __init__(self, cmp_list, data_total):
#         self.cmp_list = cmp_list
#         self.data_total = data_total
        
#         df_cis, df_bs, df_cf = self.search_table(self.cmp_list)

#         df_cis_1 = self.cis_preprocessing(df_cis)
#         df_bs_1 = self.bs_preprocessing(df_bs)
#         df_cf_1 = self.cf_preprocessing(df_cf)

#         self.total_dataframe(df_cis_1, df_bs_1, df_cf_1, cmp_list)
    
#     def search_table(self, cmp_list):
#         basic_info = crp_list.find_by_corp_name(cmp_list, exactly=True)[0]
#         fs = basic_info.extract_fs(bgn_de='20190101')

#         # 'bs' 재무상태표, 'is' 손익계산서, 'cis' 포괄손익계산서, 'cf' 현금흐름표
#         df_cis = fs['cis']
#         df_bs = fs['bs']
#         df_cf = fs['cf']

#         df_cis.to_csv('data/포괄손익계산서/' + cmp_list + ' 포괄손익계산서.csv', index=False)
#         df_bs.to_csv('data/재무상태표/' + cmp_list + ' 재무상태표.csv', index=False)
#         df_cf.to_csv('data/현금흐름도/' + cmp_list + ' 현금흐름표.csv', index=False)

#         return df_cis, df_bs, df_cf

#     def cis_preprocessing(self, df_cis):
#         df_cis_1 = df_cis.iloc[:, [1, 7, 8, 9]]
#         df_cis_1.columns = ['', df_cis_1.columns[1][0], df_cis_1.columns[2][0], df_cis_1.columns[3][0]]
#         df_cis_1 = df_cis_1.T
#         df_cis_1 = df_cis_1.rename(columns=df_cis_1.iloc[0])
#         df_cis_1 = df_cis_1.drop(df_cis_1.index[0])
#         df_cis_1 = df_cis_1.reset_index().rename(columns={'index': 'date'})
#         df_cis_1['date'] = df_cis_1['date'].str[9:]
#         print('cis_proprocessing complete')

#         return df_cis_1

#     def bs_preprocessing(self, df_bs):
#         df_bs_1 = df_bs.iloc[:, [1, 7, 8, 9]]
#         df_bs_1.columns = ['', df_bs_1.columns[1][0], df_bs_1.columns[2][0], df_bs_1.columns[3][0]]
#         df_bs_1 = df_bs_1.T
#         df_bs_1 = df_bs_1.rename(columns=df_bs_1.iloc[0])
#         df_bs_1 = df_bs_1.drop(df_bs_1.index[0])
#         df_bs_1 = df_bs_1.reset_index().rename(columns={'index': 'date'})
#         print('bs_preprocessing complete')

#         return df_bs_1

#     def cf_preprocessing(self, df_cf):
#         df_cf_1 = df_cf.iloc[:, [1, 6, 7, 8]]
#         df_cf_1.columns = ['', df_cf_1.columns[1][0], df_cf_1.columns[2][0], df_cf_1.columns[3][0]]
#         df_cf_1 = df_cf_1.T
#         df_cf_1 = df_cf_1.rename(columns=df_cf_1.iloc[0])
#         df_cf_1 = df_cf_1.drop(df_cf_1.index[0])
#         df_cf_1 = df_cf_1.reset_index().rename(columns={'index': 'date'})
#         df_cf_1['date'] = df_cf_1['date'].str[9:]
#         print('cf_preprocessing complete')

#         return df_cf_1
    
#     def total_dataframe(self, df_cis_1, df_bs_1, df_cf_1, cmp_list):
#         self.data_total = pd.merge(df_cis_1, df_bs_1, on='date')
#         self.data_total = pd.merge(self.data_total, df_cf_1, on='date')
#         self.data_total.to_csv('data/total/' + cmp_list + ' total.csv', index=False)
#         self.data_total = self.data_total.tolist()
        
#         return pd.DataFrame(self.data_total)
